# ⭐ Lora Trainer by Hollowstrawberry

This is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!


### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |
| 🌟 **Legacy XL Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) |  |

In [1]:
import os
import re
import toml
from time import time
from IPython.display import Markdown, display

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None
if "optimizer" not in globals():
  optimizer = "AdamW8bit"
if "optimizer_args" not in globals():
  optimizer_args = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""
if "weighted_captions" not in globals():
  weighted_captions = False
if "adjust_tags" not in globals():
  adjust_tags = False
if "keep_tokens_weight" not in globals():
  keep_tokens_weight = 1.0

COLAB = True # low ram
XFORMERS = True
SOURCE = "https://github.com/uYouUs/sd-scripts"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

#@title ## 🚩 從這裡開始

#@markdown ### ▶️ 設定
#@markdown 您的專案名稱將與包含您圖片的資料夾名稱相同。不允許有空格。
project_name = "mushroom" #@param {type:"string"}
project_name = project_name.strip()
#@markdown 資料夾結構並不重要，純粹是為了方便整理。請確保始終選擇相同的結構。我喜歡按專案組織。
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]
#@markdown 決定將下載並用於訓練的模型。這些選項應該會產生乾淨且一致的結果。您也可以透過貼上其下載連結來選擇您自己的模型。
training_model = "Anime (animefull-final-pruned-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "AnyLora (AnyLoRA_noVae_fp16-pruned.ckpt)", "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)"]
optional_custom_training_model_url = "" #@param {type:"string"}
custom_model_is_based_on_sd2 = False #@param {type:"boolean"}

if optional_custom_training_model_url:
  model_url = optional_custom_training_model_url
elif "AnyLora" in training_model:
  model_url = "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt"
elif "Anime" in training_model:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"
else:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors"

#@markdown ### ▶️ 處理
#@markdown Stable Diffusion 1.5 的標準解析度為 512。更高解析度的訓練會慢得多，但可以產生更好的細節。<p>
#@markdown 圖片在訓練時會自動縮放以產生最佳結果，因此您無需自己裁剪或調整大小。
resolution = 512 #@param {type:"slider", min:512, max:1024, step:128}
#@markdown 此選項將以正常和翻轉的方式訓練您的圖片，無需額外費用，以便從中學習更多。如果您的圖片少於 20 張，請特別開啟此選項。<p>
#@markdown **如果您關心 Lora 中的不對稱元素，請關閉此選項**。
flip_aug = True #@param {type:"boolean"}
#markdown 留空表示不使用標題檔。
caption_extension = ".txt" #param {type:"string"}
#@markdown 原地打亂動漫標籤可以改善學習和提示效果。激活標籤位於每個文字檔案的開頭，不會被打亂。
shuffle_tags = True #@param {type:"boolean"}
shuffle_caption = shuffle_tags
activation_tags = "1" #@param [0,1,2,3]
keep_tokens = int(activation_tags)

#@markdown ### ▶️ 步驟 <p>
#@markdown 您的圖片在訓練期間將重複此數字。我建議您的圖片數量乘以其重複次數介於 200 到 400 之間。
num_repeats = 20 #@param {type:"number"}
#@markdown 選擇您要訓練多久。一個好的起點是約 10 個 Epoch 或約 2000 個步驟。<p>
#@markdown 一個 Epoch 的步驟數等於：您的圖片數量乘以其重複次數，除以批次大小。<p>
preferred_unit = "Epochs" #@param ["Epochs", "Steps"]
how_many = 20 #@param {type:"number"}
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
#@markdown 儲存更多 Epoch 可以讓您更好地比較您的 Lora 進度。
save_every_n_epochs = 1 #@param {type:"number"}
keep_only_last_n_epochs = 10 #@param {type:"number"}
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs
#@markdown 增加批次大小可以加快訓練速度，但可能會使學習效果變差。建議使用 2 或 3。
train_batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}

#@markdown ### ▶️ 學習
#@markdown 學習率對您的結果最重要。如果您想用大量圖片進行較慢的訓練，或者如果您的 dim 和 alpha 很高，請將 unet 學習率移至 2e-4 或更低。<p>
#@markdown 文字編碼器有助於您的 Lora 稍微更好地學習概念。建議將其設定為 unet 學習率的一半或五分之一。如果您正在訓練風格，甚至可以將其設定為 0。
unet_lr = 1e-4 #@param {type:"number"}
text_encoder_lr = 5e-5 #@param {type:"number"}
#@markdown 調度器是指導學習率的演算法。如果您不確定，請選擇 `constant` 並忽略數字。我個人建議使用 `cosine_with_restarts` 並重新啟動 3 次。
lr_scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler_number = 3 #@param {type:"number"}
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
#@markdown 在訓練期間用於提高效率的學習率「熱身」步驟。我建議將其保持在 5%。
lr_warmup_ratio = 0.05 #@param {type:"slider", min:0.0, max:0.5, step:0.01}
lr_warmup_steps = 0
#@markdown 新功能，隨時間調整損失，使學習更有效率，訓練時間可縮短約一半。使用論文建議的 5.0 值。
min_snr_gamma = True #@param {type:"boolean"}
min_snr_gamma_value = 5.0 if min_snr_gamma else None

#@markdown ### ▶️ 結構
#@markdown LoRA 是經典類型，適用於各種目的。LoCon 適用於藝術風格，因為它有更多層來學習資料集的更多方面。
lora_type = "LoRA" #@param ["LoRA", "LoCon"]

#@markdown 以下是以下設定的一些建議值：

#@markdown | type | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | LoRA | 16 | 8 |   |   |
#@markdown | LoCon | 16 | 8 | 8 | 4 |

#@markdown 較高的 dim 意味著較大的 Lora，它可以包含更多信息，但更多並不總是更好。建議 dim 在 8-32 之間，alpha 等於 dim 的一半。
network_dim = 32 #@param {type:"slider", min:1, max:128, step:1}
network_alpha = 16 #@param {type:"slider", min:1, max:128, step:1}
#@markdown 以下兩個值僅適用於 LoCon 的附加層。
conv_dim = 8 #@param {type:"slider", min:1, max:64, step:1}
conv_alpha = 4 #@param {type:"slider", min:1, max:64, step:1}

network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

#@markdown ### ▶️ 準備 <p>
#@markdown 您現在可以執行此儲存格來「烹飪」您的 Lora 了。祝您好運！<p>


# 👩‍💻 Cool code goes here

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
  if override_values_for_dadapt_and_prodigy:
    unet_lr = 0.5
    text_encoder_lr = 0.5
    lr_scheduler = "constant_with_warmup"
    lr_warmup_ratio = 0.05
    network_alpha = network_dim

  if not optimizer_args:
    optimizer_args = ["decouple=True","weight_decay=0.01","betas=[0.9,0.999]"]
    if optimizer == "Prodigy":
      optimizer_args.extend(["d_coef=2","use_bias_correction=True"])
      if lr_warmup_ratio > 0:
        optimizer_args.append("safeguard_warmup=True")
      else:
        optimizer_args.append("safeguard_warmup=False")

root_dir = "/content" if COLAB else "~/Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_wrapper.py -q -O train_network_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install -U torch==2.4 xformers triton torchvision==0.19 --index-url https://download.pytorch.org/whl/cu121
  !pip install accelerate==0.25.0 transformers==4.36.2 diffusers[torch]==0.25.0 ftfy==6.1.1 \
    opencv-python==4.8.1.78 einops==0.7.0 pytorch-lightning==1.9.0 bitsandbytes==0.43.0 \
    prodigyopt==1.0 lion-pytorch==0.0.6 tensorboard safetensors==0.4.2 altair==4.2.2 \
    easygui==0.98.3 toml==0.10.2 voluptuous==0.13.1 huggingface-hub==0.20.1 imagesize==1.4.1 rich==13.7.1 numpy==1.26.4
  !pip install -e .

  # patch kohya for minor stuff
  if COLAB:
    !sed -i "s@cpu@cuda@" library/model_util.py # low ram
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, keep_tokens_weight, weighted_captions, adjust_tags
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  print("\n💿 Checking dataset...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Please choose a valid project name.")
    return

  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except Exception as e:
      print(f"💥 Error: Your custom dataset is invalid or contains an error! Please check the original template. Details: {e}")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = list(datasets_dict.keys()) # Use folders from custom_dataset

    # Validate folders and files within custom_dataset paths
    all_files_in_custom_dataset = []
    for folder in folders:
        if not os.path.exists(folder):
            print(f"💥 Error: The folder {folder.replace('/content/drive/', '')} doesn't exist as specified in custom_dataset.")
            return
        files_in_folder = os.listdir(folder)
        if not files_in_folder:
            print(f"💥 Error: Your folder {folder.replace('/content/drive/', '')} is empty as specified in custom_dataset.")
            return
        for f in files_in_folder:
            # Check if item is a file and has a supported extension (excluding directories)
            if os.path.isfile(os.path.join(folder, f)) and not f.lower().endswith((".txt", ".npz", *supported_types)):
                 print(f"💥 Error: Invalid file in dataset folder {folder.replace('/content/drive/', '')}: \"{f}\". Aborting.")
                 return
        all_files_in_custom_dataset.extend([os.path.join(folder, f) for f in files_in_folder])


    files = all_files_in_custom_dataset # Use files from custom_dataset folders
    images_repeats = {folder: (len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}

  else: # Original logic for single image folder
    reg = []
    folders = [images_folder]

    if not os.path.exists(images_folder):
      print(f"💥 Error: The folder {images_folder.replace('/content/drive/', '')} doesn't exist.")
      return

    files = os.listdir(images_folder)

    for f in files:
        if os.path.isfile(os.path.join(images_folder, f)) and not f.lower().endswith((".txt", ".npz", *supported_types)):
             print(f"💥 Error: Invalid file in dataset: \"{f}\". Aborting.")
             return

    images_in_main_folder = len([f for f in files if os.path.isfile(os.path.join(images_folder, f)) and f.lower().endswith(supported_types)])
    images_repeats = {images_folder: (images_in_main_folder, num_repeats)}

    if not images_in_main_folder:
        print(f"💥 Error: Your {images_folder.replace('/content/drive/', '')} folder is empty of supported image types.")
        return


  if not [f for f in files if os.path.isfile(f) and f.lower().endswith(".txt")]:
    caption_extension = ""
  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 Error: Invalid path to existing Lora. Example: /content/drive/MyDrive/Loras/example.safetensors")
    return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
  estimated_epochs = int(total_steps/steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
    print(f"📈 Found {img} images with {rep} repeats, equaling {img*rep} steps.")
  print(f"📉 Divide {pre_steps_per_epoch} steps by {train_batch_size} batch size to get {steps_per_epoch} steps per epoch.")
  if max_train_epochs:
    print(f"🔮 There will be {max_train_epochs} epochs, for around {total_steps} total training steps.")
  else:
    print(f"🔮 There will be {total_steps} steps, divided into {estimated_epochs} epochs and then some.")


  if total_steps > 10000:
    print("💥 Error: Your total steps are too high. You probably made a mistake. Aborting...")
    return

  if adjust_tags:
    print(f"\n📎 Weighted tags: {'ON' if weighted_captions else 'OFF'}")
    if weighted_captions:
      print(f"📎 Will use {keep_tokens_weight} weight on {keep_tokens} activation tag(s)")
    print("📎 Adjusting tags...")
    adjust_weighted_tags(folders, keep_tokens, keep_tokens_weight, weighted_captions)

  return True

def adjust_weighted_tags(folders, keep_tokens: int, keep_tokens_weight: float, weighted_captions: bool):
  weighted_tag = re.compile(r"\((.+?):[.\d]+\)(,|$)")
  for folder in folders:
    for txt in [f for f in os.listdir(folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(folder, txt), 'r') as f:
        content = f.read()
      # reset previous changes
      content = content.replace('\\', '')
      content = weighted_tag.sub(r'\1\2', content)
      if weighted_captions:
        # re-apply changes
        content = content.replace(r'(', r'\(').replace(r')', r'\)').replace(r':', r'\:')
        if keep_tokens_weight > 1:
          tags = [s.strip() for s in content.split(",")]
          for i in range(min(keep_tokens, len(tags))):
            tags[i] = f'({tags[i]}:{keep_tokens_weight})'
          content = ", ".join(tags)
      with open(os.path.join(folder, txt), 'w') as f:
        f.write(content)

def create_config():
  global dataset_config_file, config_file, model_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ 使用自訂設定檔 {config_file}")
  else:
    config_dict = {
      "additional_network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if text_encoder_lr == 0 else None,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": optimizer,
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "train_batch_size": train_batch_size,
        "noise_offset": None,
        "clip_skip": 2,
        "min_snr_gamma": min_snr_gamma_value,
        "weighted_captions": weighted_captions,
        "seed": 42,
        "max_token_length": 225,
        "xformers": XFORMERS,
        "lowram": COLAB,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": log_folder,
        "output_name": project_name,
        "log_prefix": project_name,
      },
      "model_arguments": {
        "pretrained_model_name_or_path": model_file,
        "v2": custom_model_is_based_on_sd2,
        "v_parameterization": True if custom_model_is_based_on_sd2 else None,
      },
      "saving_arguments": {
        "save_model_as": "safetensors",
      },
      "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
      },
      "dataset_arguments": {
        "cache_latents": True,
      },
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 設定檔已儲存到 {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ 使用自訂資料集設定檔 {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 資料集設定檔已儲存到 {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file
  real_model_url = model_url.strip()

  if real_model_url.lower().endswith((".ckpt", ".safetensors")):
    model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
  else:
    model_file = "/content/downloaded_model.safetensors"
    if os.path.exists(model_file):
      !rm "{model_file}"

  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", model_url):
    if m.group(2):
      model_file = f"/content{m.group(2)}.safetensors"
    if m := re.search(r"modelVersionId=([0-9]+)", model_url):
      real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
    else:
      raise ValueError("optional_custom_training_model_url contains a civitai link, but the link doesn't include a modelVersionId. You can also right click the download button to copy the direct download link.")

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"Renamed model to {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except:
      return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 連接到 Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏭 正在安裝依賴庫...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ 安裝完成，耗時 {int(t1-t0)} 秒。")
  else:
    print("\n✅ 依賴庫已安裝。")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 正在下載模型...")
    if not download_model():
      print("\n💥 錯誤：您選擇的模型無效或已損壞，或者無法下載。您可以使用 civitai 或 huggingface 連結，或任何直接下載連結。")
      return
    print()
  else:
    print("\n🔄 模型已下載。\n")

  create_config()

  print("\n⭐ 正在啟動訓練器...\n")
  os.chdir(repo_dir)

  !accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

main()

📂 連接到 Google Drive...
Mounted at /content/drive

💿 Checking dataset...
📁MyDrive/Loras/mushroom/dataset
📈 Found 13 images with 20 repeats, equaling 260 steps.
📉 Divide 260 steps by 2 batch size to get 130.0 steps per epoch.
🔮 There will be 20 epochs, for around 2600 total training steps.

🏭 正在安裝依賴庫...

Cloning into '/content/kohya-trainer'...
remote: Enumerating objects: 7040, done.
remote: Total 7040 (delta 0), reused 0 (delta 0), pack-reused 7040 (from 1)
Receiving objects: 100% (7040/7040), 10.16 MiB | 15.32 MiB/s, done.
Resolving deltas: 100% (5054/5054), done.
37 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 new

Obtaining file:///content/kohya-trainer
  Preparing metadata (setup.py) ... done
  Running setup.py develop for library


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(



✅ 安裝完成，耗時 263 秒。

🔄 正在下載模型...

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
1505b9|OK  |   117MiB/s|//content/animefull-final-pruned-fp16.safetensors

Status Legend:
(OK):download completed.


📄 設定檔已儲存到 /content/drive/MyDrive/Loras/mushroom/training_config.toml
📄 資料集設定檔已儲存到 /content/drive/MyDrive/Loras/mushroom/dataset_config.toml

⭐ 正在啟動訓練器...

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/gene

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## *️⃣ Extras

You can run these before starting the training.

In [7]:
#@markdown ### 🔮 優化器
#@markdown 如果您執行此儲存格，您將更改用於訓練的優化器。否則，預設將是推薦的 `AdamW8bit`。<p>
#@markdown * Dadapt 和 Prodigy 會自動管理學習率，對於小型資料集非常有效。您無需更改其他任何內容即可使用它們。<p>
#@markdown 對於 Dadapt 和 Prodigy，如果勾選了方框，以下值將被覆蓋：<p>
#@markdown `learning_rate=0.5`, `network_alpha=network_dim`, `lr_scheduler="constant_with_warmup"`, `lr_warmup_ratio=0.05`<p>
#@markdown 對於 Dadapt 和 Prodigy，如果 `optimizer_args` 留空，預設值將為 `decouple=True, weight_decay=0.01, betas=[0.9,0.999]`<p>
#@markdown 對於 Prodigy，還會額外包含：`d_coef=2, use_bias_correction=True, safeguard_warmup=True`<p>
optimizer = "Prodigy" #@param ["AdamW8bit", "Prodigy", "DAdaptation", "DadaptAdam", "DadaptLion", "AdamW", "Lion", "SGDNesterov", "SGDNesterov8bit", "AdaFactor"]
optimizer_args = "" #@param {type:"string"}
splitter = ", " if ", " in optimizer_args else ","
optimizer_args = [a.strip() for a in optimizer_args.split(splitter) if a]
override_values_for_dadapt_and_prodigy = True #@param {type:"boolean"}

### 📚 Multiple folders in dataset
Below is a template allowing you to define multiple folders in your dataset. You must include the location of each folder and you can set different number of repeats for each one. To add more folders simply copy and paste the sections starting with `[[datasets.subsets]]`.

When enabling this, the number of repeats set in the main cell will be ignored, and the main folder set by the project name will also be ignored.

You can make one of them a regularization folder by adding `is_reg = true`  
You can also set different `keep_tokens`, `flip_aug`, etc.

In [7]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/mushroom/dataset/good_images"
num_repeats = 3

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/mushroom/dataset/normal_images"
num_repeats = 1

"""

In [1]:
custom_dataset = None

In [8]:
#@markdown ### 🤓 其他
#@markdown 這些功能保留在此處，供少數使用者使用。

#@markdown 權重標題檔是一項新功能，允許您使用括號來增加資料集中特定標籤的權重，與您的 webui 提示詞相同。<p>
#@markdown 標籤中正常的括號，例如 `(series names)`，需要像 `\(series names\)` 一樣進行轉義。
weighted_captions = False #@param {type:"boolean"}

#markdown 透過啟用 `adjust_tags`，您將允許此 Colab 在運行之前修改您的標籤，以根據 `weighted_captions` 的開或關自動調整。<p>
#markdown 然後，您可以增加 `activation_tag_weight` 來提高您的激活標籤的有效性。
adjust_tags = False #param {type:"boolean"}
activation_tag_weight = "1.0" #param ["1.0","1.1","1.2"]
keep_tokens_weight = float(activation_tag_weight)

#@markdown 您可以在這裡寫下您的 Google Drive 中的路徑，以載入現有的 Lora 檔案繼續訓練。<p>
#@markdown **警告：** 這與一次長時間的訓練會話不同。Epochs 會從頭開始，並且結果可能會比較差。
continue_from_lora = "" #@param {type:"string"}
if continue_from_lora and not continue_from_lora.startswith("/content/drive/MyDrive"):
  import os
  continue_from_lora = os.path.join("/content/drive/MyDrive", continue_from_lora)

In [9]:
#@markdown ### 📂 解壓縮資料集
#@markdown 將單個檔案上傳到您的 Drive 會慢得多，因此如果您的資料集在您的電腦中，您可能需要上傳一個 zip 檔案。
zip = "/content/drive/MyDrive/my_dataset.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 連接到 Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ 完成")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/my_dataset.zip'

In [8]:
#@markdown ### 🔢 計算資料集檔案數量
#@markdown Google Drive 無法計算資料夾中的檔案數量，因此這將顯示所有資料夾和子資料夾中的檔案數量。
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 連接到 Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} 圖片 | {captions:>4} 標題檔 |"
  if tree[path] and others:
    tree[path] += f" {others:>4} 其他檔案"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))

📂 連接到 Google Drive...

Mounted at /content/drive
📁Loras/mushroom/dataset/good_images   |   13 圖片 |    0 標題檔 |


# 📈 Plot training results
You can do this after running the trainer. You don't need this unless you know what you're doing.  
The first cell below may fail to load all your logs. Keep trying the second cell until all data has loaded.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir={log_folder}/

In [ ]:
from tensorboard import notebook
notebook.display(port=6006, height=800)

# ⭐ Lora Trainer by Hollowstrawberry

這是基於 [Kohya-ss](https://github.com/kohya-ss/sd-scripts) 和 [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb) 的工作。謝謝！

### ⭕ 免責聲明
本文檔旨在研究機器學習領域的尖端技術。請閱讀並遵守 [Google Colab 指南](https://research.google.com/colaboratory/faq.html) 及其 [服務條款](https://research.google.com/colaboratory/tos_v3.html)。

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **首頁** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **資料集製作器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora 訓練器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora 訓練器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |
| 🌟 **Legacy XL 訓練器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) |  |

## *️⃣ 附加功能

這些可以在開始訓練前執行。

### 📚 資料集中的多個資料夾
以下是一個模板，允許您在資料集中定義多個資料夾。您必須包含每個資料夾的位置，並且可以為每個資料夾設定不同的重複次數。要添加更多資料夾，只需複製並貼上以 `[[datasets.subsets]]` 開頭的部分。

啟用此選項時，主儲存格中設定的重複次數將被忽略，並且由專案名稱設定的主資料夾也將被忽略。

您可以通過添加 `is_reg = true` 將其中一個設為正則化資料夾。
您也可以設定不同的 `keep_tokens`、`flip_aug` 等。

# 📈 繪製訓練結果
您可以在運行訓練器後執行此操作。除非您知道自己在做什麼，否則不需要這個。<br>
下面的第一個儲存格可能無法載入所有日誌。請不斷嘗試第二個儲存格，直到所有數據都載入完成。

# Task
Generate `.txt` caption files for all images in the directory "/content/drive/MyDrive/Loras/mushroom/dataset" using the BLIP model.

## Summary:

### Data Analysis Key Findings

*   The BLIP model (specifically the `microsoft/git-large-textcaps` model) was successfully used to generate captions for 15 images in the target directory.
*   The captioning script initially created files with a `.caption` extension, which were subsequently renamed to the required `.txt` extension.

### Insights or Next Steps

*   Ensure that the captioning script is configured to output files directly with the `.txt` extension in future runs to avoid the renaming step.


## 安裝 BLIP 依賴庫

### 子任務：
安裝使用 BLIP 模型所需的 Python 函式庫。

In [9]:
# Reasoning: 安裝 BLIP 模型所需的 Python 函式庫。
import os

# Change to the repository directory if necessary
# repo_dir = "/content/kohya-trainer"
# os.chdir(repo_dir)

# Install necessary libraries for BLIP
!pip install transformers==4.36.2 accelerate==0.25.0
!pip install Pillow>=9.0.0
!pip install --upgrade google-cloud-storage
!pip install open_clip_torch
!pip install sentencepiece
!pip install protobuf==3.20.3
!pip install timm==0.6.13
!pip install einops==0.6.1
!pip install opencv-python==4.8.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 567.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-storage-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.98.0 requires google-cloud-storage<3.0.0,>=1.32.0, but you have google-cloud-storage 3.1.1 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
google-cloud-aiplatform 1.98.0 requires google-cloud-storage<3.0.0,>=1.32.0, but you have google-cloud-storage 3.1.1 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.15
    Uninstalling timm-1.0.15:
      Successfully uninstalled timm-1.0.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: einops
    Found existing installation: einops 0.8.1
    Uninstalling einops-0.8.1:
      Successfully uninstalled einops-0.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86


## 載入 BLIP 模型

### 子任務：
載入預訓練的 BLIP 模型。

In [3]:
# Reasoning: 載入預訓練的 BLIP 模型。
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

print("BLIP 模型載入完成！")

BLIP 模型載入完成！


In [4]:
# ## 載入 BLIP 模型
# ### 子任務：
# 載入預訓練的 BLIP 模型。
# Reasoning: 重新嘗試載入預訓練的 BLIP 模型，因為之前的嘗試失敗了。
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

print("BLIP 模型載入完成！")

BLIP 模型載入完成！


## 使用 BLIP 生成並儲存標題檔 (多資料夾)

### 子任務：
遍歷指定的多個資料集資料夾中的圖片檔案，使用載入的 BLIP 模型為每張圖片生成標題檔，並將這些標題檔儲存到對應的 `.txt` 檔案。

In [4]:
# Reasoning: 遍歷指定的多個資料集資料夾中的圖片檔案，使用載入的 BLIP 模型為每張圖片生成標題檔，並將這些標題檔儲存到對應的 .txt 檔案。
import os
from PIL import Image

image_folders = [
     "/content/drive/MyDrive/Loras/mushroom/dataset"
    # "/content/drive/MyDrive/Loras/mushroom/dataset/good_images",
    # "/content/drive/MyDrive/Loras/mushroom/dataset/normal_images"
]

supported_extensions = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

for folder_path in image_folders:
    if not os.path.exists(folder_path):
        print(f"Warning: Folder not found: {folder_path}. Skipping.")
        continue

    print(f"Generating captions for images in: {folder_path}")
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(supported_extensions)]

    if not image_files:
        print(f"No supported image files found in {folder_path}. Skipping.")
        continue

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        caption_path = os.path.splitext(image_path)[0] + ".txt"

        # Check if caption file already exists
        if os.path.exists(caption_path):
            print(f"Caption file already exists for {image_file}. Skipping.")
            continue

        try:
            raw_image = Image.open(image_path).convert("RGB")

            # unconditional image captioning
            # inputs = processor(raw_image, return_tensors="pt").to("cuda") # Use this if CUDA is available and you want to use GPU
            inputs = processor(raw_image, return_tensors="pt")

            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)

            with open(caption_path, "w") as f:
                f.write(caption)

            print(f"Generated caption for {image_file}")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    print(f"Finished generating captions for {folder_path}")

print("\nAll caption generation tasks completed.")

Generating captions for images in: /content/drive/MyDrive/Loras/mushroom/dataset


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption for dgu_walk_front.png


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption for dgu_back_starbag.png


...

Generated caption for dgu_walk_left.png


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption for dgu_happy_jump_left.png


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption for dgu_happy_jump_right.png
Finished generating captions for /content/drive/MyDrive/Loras/mushroom/dataset

All caption generation tasks completed.


In [3]:
import os

dataset_base_path = "/content/drive/MyDrive/Loras/mushroom/dataset"

if os.path.exists(dataset_base_path):
    print(f"Contents of {dataset_base_path}:")
    for item in os.listdir(dataset_base_path):
        print(item)
else:
    print(f"Base dataset path not found: {dataset_base_path}")

Contents of /content/drive/MyDrive/Loras/mushroom/dataset:
good_images
normal_images
dgu_sitting_of_the_left_holding_an_egg.png 的副本
dgu_santa_gift.png 的副本
dgu_graduation_cap.png 的副本
dgu_walk_front.png 的副本
dgu_love_heart.png 的副本
dgu_pingpong.png 的副本
dgu_happy_jump_left.png 的副本
dgu_reading.png 的副本
dgu_birthday_cap_and_cake.png 的副本
dgu_crown_cap.png 的副本
dgu_back_starbag.png 的副本
dgu_happy_jump_right.png 的副本
dgu_walk_left.png 的副本


In [2]:
# ## 使用 BLIP 生成並儲存標題檔 (多資料夾)
# ### 子任務：
# 遍歷指定的多個資料集資料夾中的圖片檔案，使用載入的 BLIP 模型為每張圖片生成標題檔，並將這些標題檔儲存到對應的 `.txt` 檔案。
# Reasoning: 遍歷指定的多個資料集資料夾中的圖片檔案，使用載入的 BLIP 模型為每張圖片生成標題檔，並將這些標題檔儲存到對應的 .txt 檔案。
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# Ensure model and processor are loaded if not already
if 'processor' not in globals() or 'model' not in globals():
    try:
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
        print("BLIP 模型已載入。")
    except Exception as e:
        print(f"無法載入 BLIP 模型：{e}")
        # Exit if model cannot be loaded
        exit()


image_folders = [
     "/content/drive/MyDrive/Loras/mushroom/dataset"#,
    # "/content/drive/MyDrive/Loras/mushroom/dataset/good_images",
    # "/content/drive/MyDrive/Loras/mushroom/dataset/normal_images"
]

supported_extensions = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

for folder_path in image_folders:
    if not os.path.exists(folder_path):
        print(f"Warning: Folder not found: {folder_path}. Skipping.")
        continue

    print(f"Generating captions for images in: {folder_path}")
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(supported_extensions)]

    if not image_files:
        print(f"No supported image files found in {folder_path}. Skipping.")
        continue

    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        caption_path = os.path.splitext(image_path)[0] + ".txt"

        # Check if caption file already exists
        if os.path.exists(caption_path):
            print(f"Caption file already exists for {image_file}. Skipping.")
            continue

        try:
            raw_image = Image.open(image_path).convert("RGB")

            # unconditional image captioning
            # inputs = processor(raw_image, return_tensors="pt").to("cuda") # Use this if CUDA is available and you want to use GPU
            inputs = processor(raw_image, return_tensors="pt")

            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)

            with open(caption_path, "w") as f:
                f.write(caption)

            print(f"Generated caption for {image_file}")

        except Exception as e:
            print(f"Error processing {image_file}: {e}")

    print(f"Finished generating captions for {folder_path}")

print("\nAll caption generation tasks completed.")

Generating captions for images in: /content/drive/MyDrive/Loras/mushroom/dataset
No supported image files found in /content/drive/MyDrive/Loras/mushroom/dataset. Skipping.

All caption generation tasks completed.
